In [1]:
import pandas as pd
import numpy as np
import os
from env import get_db_url
from pathlib import Path 
import csv
import acquire

#from env import user, password, host

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
sql_query = """SELECT bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet, yearbuilt, taxvaluedollarcnt,taxamount,fips
                FROM properties_2017
                LEFT JOIN propertylandusetype as pl using (propertylandusetypeid)
                WHERE pl.propertylandusetypeid = '261'"""

df = pd.read_sql(sql_query, get_db_url('zillow'))



In [ ]:
#SAVE QUERY TO CSV
filepath = Path('zillow.csv')
filepath.parent.mkdir(parents=True,exist_ok=True)
df.to_csv(filepath, index = False)

In [ ]:
#reading dataframe from local drive
df = pd.read_csv('zillow.csv')
df

In [ ]:
# Basic information about table, how many nulls are in each column 

df.info(show_counts = True)

In [ ]:
# rename columns that can be changed for readability
df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms', 
                          'calculatedfinishedsquarefeet':'area',
                          'taxvaluedollarcnt':'tax_value', 
                          'yearbuilt':'year_built'})

In [ ]:
# get total of null values for each row
df.isnull().sum()

**Takeaway**
- If our plan is to remove outliers, could removing outliers also reduce our null count?

### Visualizing Distributions & Outliers

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in df.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):

    # i starts at 0, but plot numbers should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    plt.ticklabel_format(useOffset=False)
    
plt.show()

**Takeaways**
- Outliers must exist in each column for the x-axis to have the range shown
- `tax_value` is looking like it has severe skew, due to a x-axis range that goes to 1e8

In [ ]:
# Looking at tax_value in closer detail
df['tax_value'].hist(bins=100)

### Boxplots
Show outliers as black diamonds

In [ ]:
#boxplot figure created
plt.figure(figsize=(8,4))

plt.ticklabel_format(useOffset=False, style='plain')
sns.boxplot(data=df.drop(columns=['fips']))

plt.show()

`tax_value` shows an extreme number of outliers making it difficult to identify outliers in other columns

It may be beneficial to make this chart larger or break it out into individual box plots

In [ ]:
# Creating boxplot for every column except fips and year built
cols = [col for col in df.columns if col not in ['fips', 'year_built']]
plt.figure(figsize=(16, 20))
for i, col in enumerate(cols):

    # i starts at 0, but plot numbers should start at 1
    plot_number = i + 1 

    # Create subplot
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[col])

    # Hide gridlines.
    plt.grid(False)

plt.show()

## Takeaways
Can see there are significant amounts of outliers in each column 

What are the quartiles for each of these columns?

In [ ]:
# showing quartiles for each column
df.describe().T

**Takeaways**
- Substantial outliers exist
- Depending on what model is being used to predict it may be beneficial to drop all nulls.
- If model will predict median value of homes than dropping nulls is not an issue. 

In [ ]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:
        
        # get quartiles
        q1, q3 = df[col].quantile([.25, .75])  
        
        # calculate interquartile range
        iqr = q3 - q1   
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [ ]:
#use 1.5 for k(1.5 times interquartile for upper and lower bound, most data lies within 1,2,3 std dev of mean
df = remove_outliers(df, 1.5, ['bedrooms', 'bathrooms', 'area', 'tax_value', 'taxamount'])
df

### Revisualize Distributions
Lets revisualize our data now that its be cleaned a bit (approximately 300,000 observations removed):

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in df.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    df[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    plt.ticklabel_format(useOffset=False)
    
    # mitigate overlap
    plt.tight_layout()
    
plt.show()

## Takeaways 
* Bedrooms and bathrooms are not normally distributed
* Other fields are not normal and are skewed

In [ ]:
# List of columns
cols = ['bedrooms', 'bathrooms', 'area', 'tax_value', 'taxamount']

plt.figure(figsize=(16, 3))

for i, col in enumerate(cols):

    # i starts at 0, but plot should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=df[[col]])

    # Hide gridlines.
    plt.grid(False)

    # sets proper spacing between plots
    plt.tight_layout()
    
plt.show()

## Takeaways 
* Still a few outliers despite the relatively restrictive 1.5 * IQR setting for the upper/lower boundary
* No need to remove these outliers

### How do the null values look now?

In [ ]:
# get total of null values for each row
df.isnull().sum()

In [ ]:
# consider imputing year ()built
df.year_built.value_counts()

In [ ]:
#glance at the data
#df.year_built.describe()

#formats data to be more readable/ multiplies all data by 10**3
df.year_built.describe().apply(lambda x: format(x, 'f'))

* The null values have been cleaned up by removing the outliers
* The mean, median, and mode are all relatively similar to each other
* We could try and develop a complex imputation method to estimate the year built based off other columns, but for speed, we will just use the median.

### If removing the outliers does not clean up nulls? What can be done? 

`.dropna()`
We can utilize some of the parameters of .dropna() to clean up our nulls:

- axis: {0 or ‘index’, 1 or ‘columns’}, default 0
    - 0, or ‘index’ : Drop rows which contain missing values.
    - 1, or ‘columns’ : Drop columns which contain missing value.
- how: {‘any’, ‘all’}, default ‘any’
    - ‘any’ : If any NA values are present, drop that row or column.
    - ‘all’ : If all values are NA, drop that row or column.
- thresh: int, optional
    - Require that many non-NA values.
- subset: array-like, optional
    - Give the columns to consider, ignore non-listed columns

Thresh is useful, but it is limited to an integer amount. What if we wanted to remove columns that had a certain proportion of na values?

In [ ]:
# Set our null threshold. Any columns that have this ratio or higher will be removed
missing_perc_thresh = 0.98

# Create empty list to keep track of which columns we plan on dropping
exclude_missing = []

# Find columns that have a greater null percentage than our threshold
num_rows = df.shape[0]
for c in df.columns:
    num_missing = df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print("We exclude: %s" % exclude_missing)

# Drop these columns from our dataset
df.drop(columns=exclude_missing, inplace=True)

### Are the column data types correct?

In [ ]:
df.info()

In [ ]:
# get value counts and decide on data types
cols = df.columns

for col in cols:
    
    print(col.upper())
    print(df[col].value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print()

* casting fips and year built as an object because it is a numerical represintation of a catagorical value
* leaving the rest as floats

In [ ]:
df.fips = df.fips.astype(object)
df.year_built = df.year_built.astype(object)

In [ ]:
df.info()

## Target Leakage

Consider the goal of the model to be developed:

        Build an end-to-end project in which you use some of their Kaggle data to predict property values

`taxamount` is determined by a real estate tax appraisers valuation of the home combined with the local tax rate.

The question to consider, does this represent information that we wouldn't have at the time of prediction? Does this represent the target variable? This is where specific domain knowledge is very important.

Tax appraised values often deviate from the sale price of a property due to a number of reasons:
- Unlike market value, homeowners are incentivized to reduce their tax value appraisal. As a result, some homeowners may misrepresent the features/size of their property to avoid a bigger tax bill
- Some counties limit the tax appraisal increase to a set amount, causing tax appraised value to significantly lag behind home values in hot markets
- Local tax appraisers evaluate homes based on evaluation criteria that can differ from county to county or even year to year based on political and administrative pressures

This data is a snapshot of home information in 2017. There is an additional column in the database containing assessment year. This might be useful to know the recency of any given taxamount value. 

When a home is being appraised to evaluate for market pricing, prior years home value is not considered in that price determination. That doesn't mean that prior years home value is not predictive of price. It could be. For this reason, we will allow `taxamount` to remain in the dataset. 

Without this specific domain knowledge, this column seems to represent the risk of target leak. If you are in a situation where you don't know if you have a target leak situation, its probably generally better to err on the side of caution.

## Split the data

In [ ]:
train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

## Impute year_built with mode

In [ ]:
#imputing year_built with mode of median for year built 
imputer = SimpleImputer(strategy='median')  # build imputer

imputer.fit(train[['year_built']]) # fit to train

# transform the data
train[['year_built']] = imputer.transform(train[['year_built']])
validate[['year_built']] = imputer.transform(validate[['year_built']])
test[['year_built']] = imputer.transform(test[['year_built']])

In [ ]:
train.isnull().sum()

In [ ]:
validate.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.head()

In [ ]:
df.size

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
'''Wrangles data from Zillow Database'''

##################################################Wrangle.py###################################################

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from env import user, password, host

#**************************************************Acquire*******************************************************

def acquire_zillow():
    ''' Acquire data from Zillow using env imports and rename columns'''
    
    url = f"mysql+pymysql://{user}:{password}@{host}/zillow"
    
    query = """
            
    SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
    FROM properties_2017

    LEFT JOIN propertylandusetype USING(propertylandusetypeid)

    WHERE propertylandusedesc IN ("Single Family Residential",                       
                                  "Inferred Single Family Residential")"""

    # get dataframe of data
    df = pd.read_sql(query, url)
    
    
    # renaming column names to one's I like better
    df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                              'bathroomcnt':'bathrooms', 
                              'calculatedfinishedsquarefeet':'area',
                              'taxvaluedollarcnt':'tax_value', 
                              'yearbuilt':'year_built',})
    return df

#**************************************************Remove Outliers*******************************************************

def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

#**************************************************Distributions*******************************************************

def get_hist(df):
    ''' Gets histographs of acquired continuous variables'''
    
    plt.figure(figsize=(16, 3))

    # List of columns
    cols = [col for col in df.columns if col not in ['fips', 'year_built']]

    for i, col in enumerate(cols):

        # i starts at 0, but plot nos should start at 1
        plot_number = i + 1 

        # Create subplot.
        plt.subplot(1, len(cols), plot_number)

        # Title with column name.
        plt.title(col)

        # Display histogram for column.
        df[col].hist(bins=5)

        # Hide gridlines.
        plt.grid(False)

        # turn off scientific notation
        plt.ticklabel_format(useOffset=False)

        plt.tight_layout()

    plt.show()
        
        
def get_box(df):
    ''' Gets boxplots of acquired continuous variables'''
    
    # List of columns
    cols = ['bedrooms', 'bathrooms', 'area', 'tax_value', 'taxamount']

    plt.figure(figsize=(16, 3))

    for i, col in enumerate(cols):

        # i starts at 0, but plot should start at 1
        plot_number = i + 1 

        # Create subplot.
        plt.subplot(1, len(cols), plot_number)

        # Title with column name.
        plt.title(col)

        # Display boxplot for column.
        sns.boxplot(data=df[[col]])

        # Hide gridlines.
        plt.grid(False)

        # sets proper spacing between plots
        plt.tight_layout()

    plt.show()
        
#**************************************************Prepare*******************************************************

def prepare_zillow(df):
    ''' Prepare zillow data for exploration'''

    # removing outliers
    df = remove_outliers(df, 1.5, ['bedrooms', 'bathrooms', 'area', 'tax_value', 'taxamount'])
    
    # get distributions of numeric data
    get_hist(df)
    get_box(df)
    
    # converting column datatypes
    df.fips = df.fips.astype(object)
    df.year_built = df.year_built.astype(object)
    
    # train/validate/test split
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
    
    # impute year built using mode
    imputer = SimpleImputer(strategy='median')

    imputer.fit(train[['year_built']])

    train[['year_built']] = imputer.transform(train[['year_built']])
    validate[['year_built']] = imputer.transform(validate[['year_built']])
    test[['year_built']] = imputer.transform(test[['year_built']])       
    
    return train, validate, test    


#**************************************************Wrangle*******************************************************


def wrangle_zillow():
    '''Acquire and prepare data from Zillow database for explore'''
    train, validate, test = prepare_zillow(acquire_zillow())
    
    return train, validate, test

In [ ]:
train, validate, test = wrangle_zillow()

In [ ]:
train.head()

In [ ]:
df = pd.read_csv("zillow.csv")
df

In [3]:
acquire.get_zillow_data()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt,taxvaluedollarcnt,taxamount,fips
0,0.0,0.0,NaN,NaN,27516.0,NaN,6037.0
1,0.0,0.0,NaN,NaN,10.0,NaN,6037.0
2,0.0,0.0,NaN,NaN,10.0,NaN,6037.0
3,0.0,0.0,NaN,NaN,2108.0,174.21,6037.0
4,4.0,2.0,3633.0,2005.0,296425.0,6941.39,6037.0
...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,2015.0,960756.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,2014.0,536061.0,6244.16,6059.0
2152860,0.0,0.0,NaN,NaN,208057.0,5783.88,6059.0
2152861,3.0,2.5,1974.0,2015.0,424353.0,5302.70,6059.0
